In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
# Config plt.
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
plt.style.use('ggplot')
rcParams['figure.figsize'] = 8, 5

# Ignore warnings.
import warnings
warnings.filterwarnings("ignore")

from plotnine import ggplot, aes, geom_density, geom_line, geom_point, geom_boxplot, geom_histogram, facet_grid, ggtitle

from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector as selector
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

Similar to Abdulrahman's data cleaning with some changes (e.g., adding datetime columns). The output is saved as cleaned_data_mohammed.csv

In [20]:
df = pd.read_excel('../data/Data_Train.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [3]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


### Data Cleaning

In [3]:
# Make feature names lower case.
df.columns = df.columns.str.lower()

# Change "data_of_journey" type to datetime.
# df.date_of_journey = pd.to_datetime(df.date_of_journey, infer_datetime_format=True)

# Create "year", "month", "day" features.
df['year'] = pd.DatetimeIndex(df.date_of_journey).year
df['month'] = pd.DatetimeIndex(df.date_of_journey).month
df['day'] = pd.DatetimeIndex(df.date_of_journey).day

# Print dataframe updated shape.
print(f'Updated dataframe shape: {df.shape}')

Updated dataframe shape: (10683, 14)


In [4]:
df['duration']

KeyError: 'duration'

In [21]:
# Convert "duration to seconds"
df.Duration = df.Duration.apply(lambda x: f'{x} 0m' if not 'm' in x else x)
df.Duration = df.Duration.apply(lambda x: f'0h {x}' if not 'h' in x else x)
df.Duration = df.Duration.apply(lambda x: int(x.split()[0][:-1])*3600 + int(x.split()[1][:-1])*60)
df.Duration.dtype

dtype('int64')

In [5]:
# Find "year" value counts.
print(df.year.value_counts())

# Drop "year" since its value is the same for all entires.
df.drop(columns=['year'], axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)

# Print dataframe updated shape.
print(f'Updated dataframe shape: {df.shape}')

2019    10683
Name: year, dtype: int64
Updated dataframe shape: (10683, 13)


In [22]:
print(df.Additional_Info.value_counts(), end='\n\n')

# Modify "additional_info" value "No Infor" to "No info"
df.Additional_Info = df.Additional_Info.str.replace('No Info', 'No info')

# Print dataframe updated shape.
print(f'Updated dataframe shape: {df.shape}')

No info                         8345
In-flight meal not included     1982
No check-in baggage included     320
1 Long layover                    19
Change airports                    7
Business class                     4
No Info                            3
1 Short layover                    1
Red-eye flight                     1
2 Long layover                     1
Name: Additional_Info, dtype: int64

Updated dataframe shape: (10683, 11)


In [23]:
# Check for Null values.
print(df.isna().sum(), end='\n\n')
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

# Print dataframe updated shape.
print(f'Updated dataframe shape: {df.shape}')

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

Updated dataframe shape: (10682, 11)


In [24]:
# Check for duplicated entries.
print(f'Total number of duplicated entries: {df.duplicated().sum()}', end='\n\n')

# Drop douplicated entires.
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

# Print updated dataframe shape.
print(f'Updated dataframe shape: {df.shape}')

Total number of duplicated entries: 220

Updated dataframe shape: (10462, 11)


In [25]:
# Modify "total_stops" values to contain only the number.
df.Total_Stops = df.Total_Stops.apply(lambda x: int(x.split()[0]) if not 'non-stop' in x.split()[0] else 0)
print(df.Total_Stops.dtype, end='\n\n')

# Print dataframe updated shape.
print(f'Updated dataframe shape: {df.shape}')

int64

Updated dataframe shape: (10462, 11)


In [26]:
# Categorize "dep_time"and "arrival_time".
def categorizee_time(input):
    time = input.split()[0]
    if int(time[:2]) >= 0 and int(time[:2]) < 6:
        return 'mid_night'
    elif int(time[:2]) >= 6 and int(time[:2]) < 12:
        return 'morning'
    elif int(time[:2]) >= 12 and int(time[:2]) < 18:
        return 'afternoon'
    elif int(time[:2]) >= 18 and int(time[:2]) < 24:
        return 'evening'

df['Dep_Dime_Cat'] = df.Dep_Time.apply(categorizee_time)
df['Arrival_Time_Cat'] = df.Arrival_Time.apply(categorizee_time)

In [11]:
df.drop(columns=['date_of_journey'], axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)

# Print dataframe updated shape.
print(f'Updated dataframe shape: {df.shape}')

Updated dataframe shape: (10462, 14)


In [12]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Dep_Dime_Cat,Arrival_Time_Cat
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,10200,0,No info,3897,evening,mid_night
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,26700,2,No info,7662,mid_night,afternoon
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,68400,2,No info,13882,morning,mid_night
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,19500,1,No info,6218,evening,evening
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,17100,1,No info,13302,afternoon,evening


In [27]:
df['Dep_Time'] =  df.Dep_Time.apply(str.split).apply(lambda x: x[0] if len(x)>=1 else None)
df['Arrival_Time'] =  df.Arrival_Time.apply(str.split).apply(lambda x: x[0] if len(x)>=1 else None)

In [28]:
df['Total_Stops'] = df['Total_Stops'].apply(str)

In [29]:


def make_datetime(date_s:pd.Series, time_s:pd.Series):
    lst = []
    for i in range(len(date_s)):
        date = date_s[i]
        time = time_s[i]
        date_time = f"{date} {time}"
        datetime = pd.to_datetime(date_time, infer_datetime_format=True)
        lst.append(datetime)
    return pd.Series(lst)


In [30]:
df['Datetime_Dep'] = make_datetime(df.Date_of_Journey, df.Dep_Time)
df['Datetime_Arrival'] = make_datetime(df.Date_of_Journey, df.Arrival_Time)

In [31]:
df.rename(mapper={"Dep_Dime_Cat": "Dep_Time_Cat"}, inplace=True, axis=1)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10462 entries, 0 to 10461
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Airline           10462 non-null  object        
 1   Date_of_Journey   10462 non-null  object        
 2   Source            10462 non-null  object        
 3   Destination       10462 non-null  object        
 4   Route             10462 non-null  object        
 5   Dep_Time          10462 non-null  object        
 6   Arrival_Time      10462 non-null  object        
 7   Duration          10462 non-null  int64         
 8   Total_Stops       10462 non-null  object        
 9   Additional_Info   10462 non-null  object        
 10  Price             10462 non-null  int64         
 11  Dep_Time_Cat      10462 non-null  object        
 12  Arrival_Time_Cat  10462 non-null  object        
 13  Datetime_Dep      10462 non-null  datetime64[ns]
 14  Datetime_Arrival  1046

In [34]:
df.drop(labels=['Date_of_Journey', 'Dep_Time', 'Arrival_Time'], axis=1, inplace=True)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10462 entries, 0 to 10461
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Airline           10462 non-null  object        
 1   Source            10462 non-null  object        
 2   Destination       10462 non-null  object        
 3   Route             10462 non-null  object        
 4   Duration          10462 non-null  int64         
 5   Total_Stops       10462 non-null  object        
 6   Additional_Info   10462 non-null  object        
 7   Price             10462 non-null  int64         
 8   Dep_Time_Cat      10462 non-null  object        
 9   Arrival_Time_Cat  10462 non-null  object        
 10  Datetime_Dep      10462 non-null  datetime64[ns]
 11  Datetime_Arrival  10462 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(8)
memory usage: 980.9+ KB


In [36]:
# Save the dataframe into a csv file.
df.to_csv('../data/cleaned_data_mohammed.csv', index=False)